In [1]:
import trimesh as tm
from pymcfs.mesh import MeshManager
from pymcfs.mcf import mean_curvature_flow


In [13]:

# 1) Create a simple demo mesh: a sphere
# mesh = tm.primitives.Sphere(radius=1.0, subdivisions=3)
mesh = tm.load("C:/Users/MainUser/Documents/GitHub/pymcfs/data/mesh/processed/TS2_wrapped.obj")

# 2) Visualize the input mesh
mm_in = MeshManager(mesh, verbose=True)
print(mm_in.analyze_mesh())
mm_in.visualize_mesh_3d(
    title="Input Mesh (Sphere)",
    backend="auto",          # "auto" tries Plotly first, then falls back to Matplotlib
    show_axes=True,
    show_wireframe=False,
)

{'face_count': 3506, 'vertex_count': 1753, 'bounds': [[8690.8955078, 2649.7304688, 2274.7929688], [9000.6582031, 2922.3132324, 2735.3129883]], 'is_watertight': True, 'is_winding_consistent': True, 'issues': [], 'volume': np.float64(3978227.863038186), 'euler_characteristic': 0, 'genus': 1, 'normal_stats': {'mean': [-0.010258101833406369, 0.0001888035639508897, 0.0001336050920683351], 'std': [0.5969021001467615, 0.622611332977133, 0.5059226509442832], 'sum': [-35.964905027922725, 0.6619452952118193, 0.4684194527915828]}, 'normal_direction': 'outward', 'duplicate_vertices': 0, 'degenerate_faces': 0, 'component_count': 1}


In [37]:

# 3) Apply ONE shrink (MCF) step
mcf_res = mean_curvature_flow(
    mesh,
    dt=0.1,
    iterations=1000,
    laplacian_type="cotangent",
    verbose=True,
)

# 4) Build a new mesh from the shrunk vertices and visualize it
mesh_shrunk = tm.Trimesh(vertices=mcf_res.vertices, faces=mesh.faces, process=False)
mm_out = MeshManager(mesh_shrunk, verbose=True)
print(mm_out.analyze_mesh())
mm_out.visualize_mesh_3d(
    title="After One MCF Step",
    backend="auto",
    show_axes=True,
    show_wireframe=False,
)

{'face_count': 3506, 'vertex_count': 1753, 'bounds': [[8702.339789783973, 2660.4819806408464, 2280.5563338171246], [8990.680177536678, 2914.9282810688214, 2731.672945883435]], 'is_watertight': True, 'is_winding_consistent': True, 'issues': [], 'volume': np.float64(3591026.047243096), 'euler_characteristic': 0, 'genus': 1, 'normal_stats': {'mean': [-0.02193656974485374, -0.0073523045132126595, 0.00622903550743723], 'std': [0.6046187444305592, 0.636251932041058, 0.4785870688809476], 'sum': [-76.90961352545722, -25.777179623323583, 21.83899848907493]}, 'normal_direction': 'outward', 'duplicate_vertices': 0, 'degenerate_faces': 0, 'component_count': 1}
